In [90]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 12 11:49:08 2019
@author: wangxinyi
"""
import pymysql
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os
import math

from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from dateutil import parser
from scipy import stats  
from scipy.stats import entropy
from sklearn.preprocessing import StandardScaler

conn= pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='root123',charset='utf8',db='car_evaluation')

all_data_df=pd.read_sql_query('''
select id,ty,concat(substr(t,1,9),'0:00') as t, max(x) as x,max(y) as y,ROUND(avg(v),2) as v,ROUND(avg(d),0) as d  , concat(max(x),'-',max(y))  as xy
from tianchi_01_train  group by id,ty,concat(substr(t,1,9),'0:00') ;
''',conn)
# select id,x,y,v,d,t,ty,concat(x,'-',y)  as xy from tianchi_01_train  order by  id,t


In [91]:
all_data_df

,id,ty,t,x,y,v,d,xy
0,0,拖网,1107 12:00:00,6.118352e+06,5.130672e+06,0.00,0.0,6118351.75770008-5130672.32583525
1,0,拖网,1107 12:10:00,6.118352e+06,5.130672e+06,0.00,0.0,6118351.75770008-5130672.32583525
2,0,拖网,1107 12:20:00,6.118352e+06,5.130672e+06,0.00,0.0,6118351.75770008-5130672.32583525
3,0,拖网,1107 12:30:00,6.118352e+06,5.130672e+06,0.00,0.0,6118351.75770008-5130672.32583525
4,0,拖网,1107 12:40:00,6.118352e+06,5.130672e+06,0.00,0.0,6118351.75770008-5130672.32583525
5,0,拖网,1107 12:50:00,6.118352e+06,5.130672e+06,0.00,0.0,6118351.75770008-5130672.32583525
6,0,拖网,1107 13:00:00,6.118352e+06,5.130672e+06,0.00,0.0,6118351.75770008-5130672.32583525
7,0,拖网,1107 13:10:00,6.118352e+06,5.130672e+06,0.00,0.0,6118351.75770008-5130672.32583525
8,0,拖网,1107 13:40:00,6.118352e+06,5.130672e+06,0.02,0.0,6118351.75770008-5130672.32583525
9,0,拖网,1107 13:50:00,6.118352e+06,5.130672e+06,0.05,0.0,6118351.75770008-5130672.32583525


In [92]:
def t_p(t):
    #return t.size
    return (parser.parse(max(t))-parser.parse(min(t))).total_seconds()/t.size

def entropy_wxy(x):
    index_x={}
    for c in x:
        if c in index_x:
            index_x[c]=index_x[c]+1
        else:
            index_x[c]=1
    vv=np.array(list(index_x.values()))    
    return entropy(vv, base=2)
def sub_num(x):
    return np.max(x)-np.min(x)

def distinct_count(xy):
    return len(set(xy))

def model_num(x):
    return np.mean(x.mode())

def month(x):
    #print(type(x))
    xx=[]
    for c in x:
        xx.append(c[0:2])
    return int(pd.Series(xx).mode()[0])

def day(x):
    #print(type(x))
    xx=[]
    for c in x:
        xx.append(c[5:7])
    return int(pd.Series(xx).mode()[0])

def distance_median(xy):
    test_f=[ [float(xx) for xx in  x.split("-")] for x in xy]
    ss=[]
    for i in range(len(test_f)-1):
        s=np.linalg.norm(np.array(test_f[i])-np.array(test_f[i+1]))  
        ss.append(s)
    return np.percentile(ss,0.5)

def per25(x):
    return np.percentile(x,0.25)

def per75(x):
    return np.percentile(x,0.75)

def kmeans_xy(xy):
    return xy.size



all_df=all_data_df.groupby(['id', 'ty']).agg({'x': [np.mean, np.max,np.min,np.median,model_num,sub_num,per25,per75],
                                              'y': [np.mean, np.max,np.min,np.median,model_num,sub_num,per25,per75],
                                              'v':[np.mean,np.std,np.median,entropy_wxy,model_num,distinct_count,per25,per75] ,
                                              'd':[np.mean,np.std,np.median,entropy_wxy,model_num,distinct_count,per25,per75],
                                              't':[t_p,month,day],
                                              'xy':[distinct_count,distance_median]
                                      })

In [10]:
all_df.head(1)

x                                                          \
               mean          amax          amin        median     model_num   
id ty                                                                         
0  拖网  6.119351e+06  6.152038e+06  6.118352e+06  6.118352e+06  6.118352e+06   

                                                            y                \
            sub_num         per25         per75          mean          amax   
id ty                                                                         
0  拖网  33686.667453  6.118352e+06  6.118352e+06  5.130494e+06  5.130781e+06   

            ...                 d                                       \
            ...       entropy_wxy model_num distinct_count per25 per75   
id ty       ...                                                          
0  拖网       ...          0.439489       0.0             15   0.0   0.0   

                   t                       xy                  
                 t_p month day distinct_count distance_median  
id ty                                                          
0  拖网  228728.818841    11  15             23             0.0  

[1 rows x 37 columns]

In [93]:
names_x_x=['c_x','min_x','max_x','median_x','model_x','sub_x','per25_x','per75_x',
           'c_y','min_y','max_y','median_y','model_y','sub_y','per25_y','per75_y',
           'avg_v','stdev_v','median_v','entropy_v','model_v','d_v','per25_v','per75_v',
           'avg_d','stdev_d','median_d','entropy_d','model_d','d_d','per25_d','per75_d',
           't','t_month','t_day',
           'd_xy','dm_xy']

names_y='ty'
all_df.columns=names_x_x

all_df=all_df.reset_index()

In [12]:
all_df.head(1)

,id,ty,c_x,min_x,max_x,median_x,model_x,sub_x,per25_x,per75_x,...,entropy_d,model_d,d_d,per25_d,per75_d,t,t_month,t_day,d_xy,dm_xy
0,0,拖网,6.119351e+06,6.152038e+06,6.118352e+06,6.118352e+06,6.118352e+06,33686.667453,6.118352e+06,6.118352e+06,...,0.439489,0.0,15,0.0,0.0,228728.818841,11,15,23,0.0


In [13]:
d = defaultdict(LabelEncoder)
df = all_df.apply(lambda x:d[x.name].fit_transform(x) if type(x[0]) is str or math.isnan(x[0])  else x)

In [94]:
names_x=['c_x','min_x','max_x','median_x','model_x','sub_x','per25_x','per75_x',
           'c_y','min_y','max_y','median_y','model_y','sub_y','per25_y','per75_y',
           'avg_v','stdev_v','median_v','entropy_v','model_v','d_v','per25_v','per75_v',
           'avg_d','stdev_d','median_d','entropy_d','model_d','d_d','per25_d','per75_d',
           't','t_month','t_day',
           'd_xy','dm_xy']

scaler = StandardScaler()
x_train= scaler.fit_transform(df[names_x])

#x_train = df.loc[:,names_x].values

y_train = df.loc[:,names_y].values

In [31]:
len(names_x)

37

In [105]:
import keras
from keras.layers import Input, Dense, Concatenate, Reshape, Dropout, merge, Add
from keras.callbacks import Callback, EarlyStopping
from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold

def NN_model():
    init = keras.initializers.glorot_uniform(seed=1)
    model = keras.models.Sequential()
    model.add(Dense(units=256, input_dim=len(names_x), kernel_initializer=init, activation='relu'))
    #model.add(Dense(units=256, kernel_initializer=init, activation='relu'))
    #model.add(Dense(units=192, kernel_initializer=init, activation='relu'))
    model.add(Dense(units=128, kernel_initializer=init, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(units=64, kernel_initializer=init, activation='relu'))
    #model.add(Dense(units=48, kernel_initializer=init, activation='relu'))
    model.add(Dense(units=32, kernel_initializer=init, activation='relu'))
    #model.add(Dropout(0.4))
    model.add(Dense(units=16, kernel_initializer=init, activation='softplus'))
    model.add(Dense(units=8, kernel_initializer=init, activation='softplus'))
    model.add(Dense(units=3, kernel_initializer=init, activation='softmax'))
    
    return model

class Metric(Callback):
    def __init__(self, model, callbacks, data):
        super().__init__()
        self.model = model
        self.callbacks = callbacks
        self.data = data

    def on_train_begin(self, logs=None):
        for callback in self.callbacks:
            callback.on_train_begin(logs)

    def on_train_end(self, logs=None):
        for callback in self.callbacks:
            callback.on_train_end(logs)

    def on_epoch_end(self, batch, logs=None):
        X_train, y_train = self.data[0][0], self.data[0][1]
        y_pred3 = self.model.predict(X_train)
        y_pred = np.zeros((len(y_pred3), ))
        y_true = np.zeros((len(y_pred3), ))
        for i in range(len(y_pred3)):
            y_pred[i] = list(y_pred3[i]).index(max(y_pred3[i]))
        for i in range(len(y_pred3)):
            y_true[i] = list(y_train[i]).index(max(y_train[i]))
        trn_s = f1_score(y_true, y_pred, average='macro')
        logs['trn_score'] = trn_s
        
        X_val, y_val = self.data[1][0], self.data[1][1]
        y_pred3 = self.model.predict(X_val)
        y_pred = np.zeros((len(y_pred3), ))
        y_true = np.zeros((len(y_pred3), ))
        for i in range(len(y_pred3)):
            y_pred[i] = list(y_pred3[i]).index(max(y_pred3[i]))
        for i in range(len(y_pred3)):
            y_true[i] = list(y_val[i]).index(max(y_val[i]))
        val_s = f1_score(y_true, y_pred, average='macro')
        logs['val_score'] = val_s
        print('trn_score', trn_s, 'val_score', val_s)

        for callback in self.callbacks:
            callback.on_epoch_end(batch, logs)

In [72]:
x_train

array([[6.11935124e+06, 6.15203843e+06, 6.11835176e+06, ...,
        1.50000000e+01, 2.30000000e+01, 0.00000000e+00],
       [6.09146039e+06, 6.10244988e+06, 6.04947187e+06, ...,
        1.80000000e+01, 1.38000000e+02, 0.00000000e+00],
       [6.18301061e+06, 6.18319102e+06, 6.18248219e+06, ...,
        1.70000000e+01, 4.00000000e+00, 0.00000000e+00],
       ...,
       [5.24022168e+06, 5.28746208e+06, 5.22870046e+06, ...,
        7.00000000e+00, 9.60000000e+01, 0.00000000e+00],
       [6.37342539e+06, 6.38762489e+06, 6.36501974e+06, ...,
        2.30000000e+01, 1.71000000e+02, 0.00000000e+00],
       [6.23542746e+06, 6.24031719e+06, 6.20637253e+06, ...,
        1.00000000e+01, 5.40000000e+01, 0.00000000e+00]])

In [95]:
from keras.utils import to_categorical

y_train = df.loc[:,names_y].values

y_train_c = to_categorical(y_train)
y_train_c[100:110]

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [106]:
n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=20200203)


import keras
from sklearn import metrics

b_size = 2
max_epochs = 200
models = []
for fold, (trn_idx, val_idx) in enumerate(kf.split(x_train, y_train)):
    print('fold:', fold)
    X_train, Y_train = x_train[trn_idx], y_train_c[trn_idx] #df[names_x].loc[trn_idx], df[names_y].loc[trn_idx] # x_train[trn_idx], y_train[trn_idx]
    X_val, Y_val = x_train[val_idx], y_train_c[val_idx]
    Y_val_o=y_train[val_idx]
    
    model = NN_model()
    simple_adam = keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy', optimizer=simple_adam, metrics=['accuracy'])
    es = EarlyStopping(monitor='val_score', patience=50, verbose=1, mode='max')#, restore_best_weights=True,)
    es.set_model(model)
    metric = Metric(model, [es], [(X_train, Y_train), (X_val, Y_val)])
    model.fit(X_train, Y_train, batch_size=b_size, epochs=max_epochs, 
              validation_data = [X_val, Y_val],
              callbacks=[metric], shuffle=True, verbose=1)
    
    models.append(model)
    test_pred = model.predict(X_val)
    test_pred_v = np.argmax(test_pred, axis=1)
        

    score_ = metrics.f1_score(Y_val_o, test_pred_v, average='macro')
    print('*'*20)
    print('total score is : {}'.format(score_))
    print('*'*20)
    break


fold: 0
Train on 5600 samples, validate on 1400 samples
Epoch 1/200
5600/5600 [==============================] - 12s 2ms/step - loss: 0.7216 - acc: 0.7139 - val_loss: 0.6569 - val_acc: 0.7464
trn_score 0.5289428945690174 val_score 0.5201597998758278
Epoch 2/200
5600/5600 [==============================] - 10s 2ms/step - loss: 0.6357 - acc: 0.7443 - val_loss: 0.6163 - val_acc: 0.7421
trn_score 0.6097360051043555 val_score 0.5854353379530344
Epoch 3/200
5600/5600 [==============================] - 10s 2ms/step - loss: 0.6013 - acc: 0.7498 - val_loss: 0.5940 - val_acc: 0.7650
trn_score 0.6340420626365154 val_score 0.6277163760918976
Epoch 4/200
5600/5600 [==============================] - 10s 2ms/step - loss: 0.5750 - acc: 0.7552 - val_loss: 0.6478 - val_acc: 0.7721
trn_score 0.6816863310885578 val_score 0.6753070508768523
Epoch 5/200
5600/5600 [==============================] - 10s 2ms/step - loss: 0.5748 - acc: 0.7495 - val_loss: 0.5651 - val_acc: 0.7486
trn_score 0.570979528889761 val_

5600/5600 [==============================] - 10s 2ms/step - loss: 0.4823 - acc: 0.7905 - val_loss: 0.5400 - val_acc: 0.7886
trn_score 0.7436196463488782 val_score 0.7135570135005268
Epoch 44/200
5600/5600 [==============================] - 10s 2ms/step - loss: 0.4651 - acc: 0.7973 - val_loss: 0.5493 - val_acc: 0.7886
trn_score 0.7377230828277703 val_score 0.699715817238152
Epoch 45/200
5600/5600 [==============================] - 10s 2ms/step - loss: 0.4672 - acc: 0.7932 - val_loss: 0.5218 - val_acc: 0.7907
trn_score 0.7462087492675272 val_score 0.7204376763833668
Epoch 46/200
5600/5600 [==============================] - 10s 2ms/step - loss: 0.4686 - acc: 0.7955 - val_loss: 0.5375 - val_acc: 0.7843
trn_score 0.7678479800014971 val_score 0.723337419177922
Epoch 47/200
5600/5600 [==============================] - 10s 2ms/step - loss: 0.4578 - acc: 0.7939 - val_loss: 0.5184 - val_acc: 0.7986
trn_score 0.7521673792850158 val_score 0.7239887132656704
Epoch 48/200
5600/5600 [================

5600/5600 [==============================] - 11s 2ms/step - loss: 0.4331 - acc: 0.8052 - val_loss: 0.5350 - val_acc: 0.7950
trn_score 0.7600683995281701 val_score 0.7158038003370427
Epoch 86/200
5600/5600 [==============================] - 11s 2ms/step - loss: 0.4260 - acc: 0.8093 - val_loss: 0.5369 - val_acc: 0.7900
trn_score 0.7436023382051173 val_score 0.7000915358317639
Epoch 87/200
5600/5600 [==============================] - 10s 2ms/step - loss: 0.4482 - acc: 0.8084 - val_loss: 0.5760 - val_acc: 0.7886
trn_score 0.722053909458401 val_score 0.6849601752342975
Epoch 88/200
5600/5600 [==============================] - 10s 2ms/step - loss: 0.4485 - acc: 0.8070 - val_loss: 0.5459 - val_acc: 0.7979
trn_score 0.7558013040629326 val_score 0.7263121255043604
Epoch 89/200
5600/5600 [==============================] - 10s 2ms/step - loss: 0.4354 - acc: 0.8102 - val_loss: 0.5758 - val_acc: 0.8057
trn_score 0.7749239744769408 val_score 0.7438695849957065
Epoch 90/200
5600/5600 [===============

5600/5600 [==============================] - 10s 2ms/step - loss: 0.4078 - acc: 0.8145 - val_loss: 0.5615 - val_acc: 0.7993
trn_score 0.7917104035113648 val_score 0.7389738039347781
Epoch 128/200
5600/5600 [==============================] - 10s 2ms/step - loss: 0.4208 - acc: 0.8086 - val_loss: 0.5895 - val_acc: 0.7993
trn_score 0.7648130740939174 val_score 0.7266119133883349
Epoch 129/200
5600/5600 [==============================] - 10s 2ms/step - loss: 0.4306 - acc: 0.8184 - val_loss: 0.7068 - val_acc: 0.7800
trn_score 0.7521571722699593 val_score 0.7205320562529424
Epoch 130/200
5600/5600 [==============================] - 10s 2ms/step - loss: 0.4546 - acc: 0.8043 - val_loss: 0.5940 - val_acc: 0.8007
trn_score 0.7811606799747284 val_score 0.7409692375586214
Epoch 131/200
5600/5600 [==============================] - 10s 2ms/step - loss: 0.4382 - acc: 0.8052 - val_loss: 0.6118 - val_acc: 0.7950
trn_score 0.7584507587730077 val_score 0.7169280897307355
Epoch 132/200
5600/5600 [=========

5600/5600 [==============================] - 10s 2ms/step - loss: 0.4359 - acc: 0.8148 - val_loss: 0.5989 - val_acc: 0.8029
trn_score 0.7699288829582848 val_score 0.7291501103524601
Epoch 170/200
5600/5600 [==============================] - 10s 2ms/step - loss: 0.4212 - acc: 0.8205 - val_loss: 0.5989 - val_acc: 0.8029
trn_score 0.7791460862544142 val_score 0.737140157686782
Epoch 00170: early stopping
********************
total score is : 0.737140157686782
********************


In [104]:
score_ = metrics.f1_score(Y_val_o, test_pred_v, average='macro')
score_
#0.7418763267415228
#0.7319544696703284

0.7168176353499538

In [52]:
all_test_df = pd.read_sql_query('''select id,x,y,v,d,t,concat(x,'-',y)  as xy from tianchi_01_test  order by  id,t  ''',conn)


test_df=all_test_df.groupby(['id']).agg({'x': [np.mean, np.max,np.min,np.median,model_num,sub_num,per25,per75],
                                              'y': [np.mean, np.max,np.min,np.median,model_num,sub_num,per25,per75],
                                              'v':[np.mean,np.std,np.median,entropy_wxy,model_num,distinct_count,per25,per75] ,
                                              'd':[np.mean,np.std,np.median,entropy_wxy,model_num,distinct_count,per25,per75],
                                              't':[t_p,month,day],
                                              'xy':[distinct_count,distance_median]
                                      })
test_df.columns=names_x_x
test_df=test_df.reset_index()

In [30]:
test_df

,id,c_x,min_x,max_x,median_x,model_x,c_y,min_y,max_y,median_y,...,model_v,d_v,avg_d,stdev_d,median_d,entropy_d,model_d,d_d,t,d_xy
0,7000,7.092492e+06,7.119130e+06,7.069441e+06,7.087014e+06,7.083957e+06,5.918416e+06,5.966537e+06,5.893360e+06,5.918174e+06,...,0.110,63.0,137.356568,115.674525,126.0,6.080001,0.0,147,6.091312e+06,160
1,7001,6.239019e+06,6.254992e+06,6.216428e+06,6.240004e+06,6.246625e+06,5.212498e+06,5.242126e+06,5.176732e+06,5.213941e+06,...,0.110,76.0,149.606987,104.616537,185.0,6.180437,40.0,144,1.380885e+05,367
2,7002,6.678498e+06,6.745702e+06,6.566008e+06,6.670449e+06,6.696588e+06,5.482646e+06,5.567766e+06,5.385267e+06,5.457670e+06,...,0.700,89.0,159.436585,113.566486,153.0,6.373595,50.0,163,1.540428e+05,396
3,7003,6.160467e+06,6.205663e+06,6.150675e+06,6.150875e+06,6.150875e+06,5.202435e+06,5.206247e+06,5.174729e+06,5.206031e+06,...,0.110,49.0,122.242353,119.624570,100.0,5.419250,0.0,147,2.228096e+05,71
4,7004,6.357542e+06,6.407177e+06,6.348864e+06,6.349065e+06,6.349065e+06,5.398000e+06,5.406460e+06,5.343990e+06,5.406460e+06,...,0.110,59.0,123.839196,116.631778,116.0,5.805988,0.0,158,2.379218e+05,100
5,7005,6.322045e+06,6.322055e+06,6.321954e+06,6.322055e+06,6.322055e+06,5.381925e+06,5.381943e+06,5.381833e+06,5.381942e+06,...,0.000,13.0,80.019802,109.330694,0.0,4.097435,0.0,112,1.565428e+05,4
6,7006,7.054221e+06,7.067504e+06,7.051061e+06,7.054275e+06,7.052163e+06,5.746713e+06,5.783325e+06,5.734358e+06,5.743727e+06,...,2.725,49.0,148.895570,98.147307,172.5,6.783413,178.0,141,2.996635e+05,286
7,7007,6.073201e+06,6.102651e+06,6.022657e+06,6.073013e+06,6.102649e+06,5.053933e+06,5.112762e+06,5.002226e+06,5.037612e+06,...,0.000,63.0,105.063260,106.541190,72.0,5.738276,0.0,163,5.528127e+06,280
8,7008,6.325893e+06,6.336889e+06,6.325721e+06,6.325721e+06,6.325721e+06,5.286243e+06,5.286323e+06,5.284157e+06,5.286256e+06,...,0.160,22.0,103.948837,111.857582,76.0,5.541970,0.0,187,1.468784e+05,10
9,7009,6.572146e+06,6.574011e+06,6.564846e+06,6.573241e+06,6.573241e+06,5.479199e+06,5.501269e+06,5.455010e+06,5.481878e+06,...,0.650,41.0,150.479714,103.255786,155.0,7.573890,0.0,240,1.507346e+05,148


In [75]:
# x_test = scaler.fit_transform(test_df[names_x])
x_test = test_df.loc[:,names_x].values

test_y_p= search.predict_proba(x_test)

In [54]:
x_test = test_df.loc[:,names_x].values
test_y= search.predict(x_test)
test_df['p_ty']=test_y
test_df['p_ty_str']=d['ty'].inverse_transform(test_y)
test_df['proba_0']=test_y_p[:,0]
test_df['proba_1']=test_y_p[:,1]
test_df['proba_2']=test_y_p[:,2]

In [55]:
test_df[['id','p_ty_str','p_ty','proba_0','proba_1','proba_2']]

,id,p_ty_str,p_ty,proba_0,proba_1,proba_2
0,7000,围网,1,0.029375,0.922732,0.047894
1,7001,拖网,2,0.044678,0.037702,0.917620
2,7002,围网,1,0.029657,0.885140,0.085203
3,7003,拖网,2,0.040819,0.037593,0.921587
4,7004,围网,1,0.029524,0.900886,0.069590
5,7005,拖网,2,0.449702,0.038231,0.512066
6,7006,围网,1,0.115559,0.825555,0.058886
7,7007,拖网,2,0.029209,0.037491,0.933300
8,7008,围网,1,0.047377,0.904069,0.048554
9,7009,拖网,2,0.030322,0.257067,0.712612


In [57]:
result_df=test_df[['id','p_ty_str']]
result_df.to_csv("tianchi01_result_0202_knn.csv",index=False,header=0)

In [56]:
result_df_p=test_df[['id','p_ty_str','p_ty','proba_0','proba_1','proba_2']]
result_df_p['proba_max']=result_df_p[['proba_0','proba_1','proba_2']].max(axis=1)
result_df_p=result_df_p[['id','p_ty_str','proba_max']]
result_df_p.to_csv("tianchi01_result_p_0202_knn.csv",index=False,header=0)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
